In [0]:
%pip install python-dotenv
%pip install psycopg2-binary
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
try:
    script_dir = os.getcwd()
    repo_root = os.path.abspath(os.path.join(script_dir, ".."))
    dotenv_path = os.path.join(repo_root, ".env")

    print(f"Raiz do repositório detectada em: {repo_root}")
    print(f"Tentando carregar o arquivo .env de: {dotenv_path}")

    load_dotenv(dotenv_path=dotenv_path)
    print("Arquivo .env carregado.")
except Exception as e:
    print(f"Não foi possível carregar o arquivo .env: {e}")

# Cria o schema e volume se não existir
catalog_name = "workspace"
schema_name = "landing_db"
volume_name = "landing_files"

full_schema_name = f"{catalog_name}.{schema_name}"
full_volume_name = f"{full_schema_name}.{volume_name}"
volume_path_base = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}"

print(f"Garantindo que o schema '{full_schema_name}' exista...")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {full_schema_name}")

print(f"Garantindo que o volume '{full_volume_name}' exista...")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {full_volume_name}")
print("Infraestrutura no Unity Catalog verificada/criada com sucesso.")

# --- CONFIGURAÇÃO DA CONEXÃO COM POSTGRESQL (SUPABASE) ---
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Validação para garantir que as variáveis foram carregadas
if not all([db_user, db_password, db_host, db_port, db_name]):
    raise ValueError(
        "Uma ou mais variáveis de ambiente do banco de dados não foram definidas. Verifique o arquivo .env"
    )
else:
    print("Credenciais do banco de dados carregadas com sucesso!")

jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}?sslmode=require&sslrootcert=system"

connection_properties = {
    "user": db_user,
    "password": db_password,
    "driver": "org.postgresql.Driver",
}

# --- PROCESSO DE INGESTÃO ---

# Lista de tabelas para ingestão
tables = [
    "companies",
    "company_reviews",
    "employment_types",
    "industries",
    "job_benefits",
    "job_skills",
    "jobs",
    "locations",
    "salary_ranges",
    "skills",
]

# Loop para ler cada tabela e salvar na camada Landing como CSV
for table_name in tables:
    print(f"Iniciando ingestão da tabela: {table_name}")

    try:
        # Leitura da tabela do banco de dados
        df = spark.read.jdbc(
            url=jdbc_url, table=f"public.{table_name}", properties=connection_properties
        )

        # Define o caminho de destino na camada Landing
        landing_path = f"{volume_path_base}/{table_name}"

        # Salva os dados como CSV
        df.write.format("csv").option("header", "true").mode("overwrite").save(
            landing_path
        )

        print(f"Tabela {table_name} salva em {landing_path}")

    except Exception as e:
        print(f"ERRO ao ingerir a tabela {table_name}: {e}")
        break